# 1. Install and Import Dependencies

In [43]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://download.pytorch.org/whl/cu117


In [44]:
!pip install transformers requests beautifulsoup4 pandas numpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [60]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
import numpy as np

# 2. Instantiate Model

In [61]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3. Encode and Calculate Sentiment

In [62]:
tokens = tokenizer.encode('I hated it!', return_tensors='pt')

In [63]:
tokens[0]

tensor([  101,   151, 39487, 10163, 10197,   106,   102])

In [64]:
tokenizer.decode(tokens[0])

'[CLS] i hated it! [SEP]'

In [65]:
result = model(tokens)

In [66]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.4732,  1.3655, -0.2287, -1.7810, -1.5031]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [67]:
result.logits

tensor([[ 2.4732,  1.3655, -0.2287, -1.7810, -1.5031]],
       grad_fn=<AddmmBackward0>)

In [68]:
int(torch.argmax(result.logits))+1

1

# 4. Collect Reviews

In [69]:
#r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
#soup = BeautifulSoup(r.text, 'html.parser')
#regex = re.compile('.*comment.*')
#results = soup.find_all('p', {'class':regex})
#reviews = [result.text for result in results]

In [70]:
#reviews

## Load Reviews into DataFrame and Score

In [72]:
data_path = os.path.join(os.pardir, 'raw_data', 'ruddit_comments_score.csv')
df = pd.read_csv(data_path)
df=df[df["body"]!="[deleted]"]
df.head(10)

,comment_id,body,score
0,cza1q49,> The difference in average earnings between m...,-0.083
1,cza1wdh,"The myth is that the ""gap"" is entirely based o...",-0.022
3,cza2bw8,The assertion is that women get paid less for ...,-0.146
4,cza2iji,You said in the OP that's not what they're mea...,-0.083
5,cza2jj3,>Men and women are not payed less for the same...,-0.042
6,cza31e2,"> All the wage gap is, is an average of total ...",-0.021
11,cza37ue,"No, the point is they aren't talking about dis...",-0.174
13,cza392y,So women are paid less for the same job?,-0.229
16,cza3m1b,"But obviously that doesn't make sense, does it...",0.000
17,cza3r5u,I think that Hollywood and professional sports...,0.098


In [74]:
import numpy as np
import pandas as pd

In [77]:
df['body'].iloc[0]

"> The difference in average earnings between men and women can be explained by taking into account relevant factors.\n\nSo it isn't a myth, you just feel that you can explain it."

In [79]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [88]:
sentiment_score(df['body'].iloc[4])

2

In [81]:
df['sentiment'] = df['body'].apply(lambda x: sentiment_score(x[:512]))

In [82]:
df

,comment_id,body,score,sentiment
0,cza1q49,> The difference in average earnings between m...,-0.083,3
1,cza1wdh,"The myth is that the ""gap"" is entirely based o...",-0.022,1
3,cza2bw8,The assertion is that women get paid less for ...,-0.146,2
4,cza2iji,You said in the OP that's not what they're mea...,-0.083,3
5,cza2jj3,>Men and women are not payed less for the same...,-0.042,2
...,...,...,...,...
5961,f0i0mqp,They should only censor things that talk badly...,0.064,1
5962,f80wlxq,> and one of them is a woman. \n\nOH SHIT we b...,0.458,1
5963,f8uksbp,how is this flared as US politics,-0.292,1
5964,fa6nc1r,People in Hong Kong must decide if they are go...,0.333,1


In [ ]:
df['body'].iloc[3]

# 5. Load Reviews into DataFrame and Score

In [ ]:
#df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
#df['review'].iloc[0]

In [ ]:
#def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
#sentiment_score(df['review'].iloc[1])

In [ ]:
#df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
#df

In [ ]:
#df['review'].iloc[3]